<a href="https://colab.research.google.com/github/fpromero/handball_temp/blob/master/BIDEA_Plus_Minus_Imanol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de Datos

## Leer el fichero de disco

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" succesfully'.format(
      name=fn, length=len(uploaded[fn])))

Saving 200822_BER-GRA.csv to 200822_BER-GRA (1).csv
User uploaded file "200822_BER-GRA.csv" succesfully


## Comprobar que se ha cargado correctamente

In [ ]:
import pandas as pd
import io

df_full = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), sep = '\t')
df_full


,Partido,Nr,D,Start,End,Equipo,Jugador/a,Posesion,No Posesion,Accion,Modelo Juego,Plus Minus Local,Plus Minus Visitante,Notas,Jugadoras,GF,SC Local,SC Visitante,Zona Accion,Defensa,Tactica,Portero/a,Porteria,Desequilibrio Juego,Periodo,Situaciones Especiales,Sanciones,Arbitro,Tiempo Jugado
0,200822_BER-GRA,1,NaN,00:02:13,00:02:23,GRANOLLERS,28-J.CASTAÑERA,[Error]/[Perdida]/[Pisando],False,At.Posicional,NaN,"01-A.DIVA, 04-M. FERNANDES, 06-N. TERÉS, 08-A....",NaN,NaN,NaN,NaN,NaN,NaN,[6m]/[6F],6:0,[Tr. 2:4]/[Extremo],NINGUNO,NaN,Igualdad,[1ºTiempo]/[05'],NaN,NaN,False,NaN
1,200822_BER-GRA,2,NaN,00:02:22,00:02:32,BERA BERA,03-E. ARROJERIA,[Error]/[Parada],False,Contraataque,NaN,"01-A.DIVA, 04-M. FERNANDES, 06-N. TERÉS, 08-A....",NaN,NaN,NaN,NaN,NaN,NaN,[CT]/[CT-B],Repliegue,[Contr.]/[2Oleada],01-N. WIGGINS,NaN,Igualdad,[1ºTiempo]/[05'],NaN,NaN,False,NaN
2,200822_BER-GRA,3,NaN,00:02:58,00:03:08,GRANOLLERS,98-M. CAPDEVILA,[Error]/[Perdida]/[Error Pase],False,At.Posicional,NaN,"01-A.DIVA, 04-M. FERNANDES, 06-N. TERÉS, 08-A....",NaN,NaN,NaN,NaN,NaN,NaN,[9m]/[9A],6:0,[Cruce]/[CxL],NINGUNO,NaN,Igualdad,[1ºTiempo]/[05'],NaN,NaN,False,NaN
3,200822_BER-GRA,4,NaN,00:03:05,00:03:15,BERA BERA,08-A. MENENDEZ,[Error]/[Fuera],False,Contraataque,NaN,"01-A.DIVA, 04-M. FERNANDES, 06-N. TERÉS, 08-A....",NaN,NaN,NaN,NaN,NaN,NaN,[CT]/[CT-B],Repliegue,[Contr.]/[2Oleada],01-N. WIGGINS,NaN,Igualdad,[1ºTiempo]/[05'],NaN,NaN,False,NaN
4,200822_BER-GRA,5,NaN,00:03:44,00:03:54,GRANOLLERS,51-J.SOBREPERA,[Error]/[Parada],False,At.Posicional,NaN,"01-A.DIVA, 04-M. FERNANDES, 06-N. TERÉS, 08-A....",NaN,NaN,NaN,NaN,NaN,NaN,[9m]/[9A],6:0,Penetracion,01-A.DIVA,NaN,Igualdad,[1ºTiempo]/[05'],NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,200822_BER-GRA,169,NaN,01:23:33,01:23:43,BERA BERA,13-L.HERNANDEZ,[Acierto]/[Gol],False,Contraataque,NaN,"08-A. MENENDEZ, 03-E. ARROJERIA, 63-M. MORENO,...",NaN,NaN,NaN,NaN,08-A. MENENDEZ:Asistencia,NaN,[CT]/[CT-A],Repliegue,[Contr.]/[1Oleada],16-M. MERA,NaN,Inferioridad,[2ºTiempo]/[60'],NaN,NaN,False,NaN
169,200822_BER-GRA,170,NaN,01:24:06,01:24:16,GRANOLLERS,09-L.KOFLER,[Acierto]/[Gol],False,At.Posicional,NaN,"08-A. MENENDEZ, 03-E. ARROJERIA, 63-M. MORENO,...",NaN,NaN,NaN,NaN,NaN,NaN,[6m]/[6A],5:1,[Juego sin Portero]/[6x6],87-R. LAIS,NaN,Igualdad,[2ºTiempo]/[60'],Tiempo Exclusión,NaN,False,NaN
170,200822_BER-GRA,171,NaN,01:24:20,01:24:30,BERA BERA,03-E. ARROJERIA,[Acierto]/[7m],True,Saque Gol,NaN,"08-A. MENENDEZ, 03-E. ARROJERIA, 63-M. MORENO,...",NaN,NaN,NaN,NaN,NaN,NaN,[CT]/[CT-B],Repliegue,Saque GOL,NINGUNO,NaN,Igualdad,[2ºTiempo]/[60'],NaN,NaN,False,NaN
171,200822_BER-GRA,172,NaN,01:24:49,01:24:59,BERA BERA,05-M. PIZZO,[Acierto]/[Gol],False,Saque Gol,NaN,"08-A. MENENDEZ, 03-E. ARROJERIA, 63-M. MORENO,...",NaN,NaN,NaN,NaN,03-E. ARROJERIA:Recibe 7m,NaN,7m,Repliegue,Saque GOL,01-N. WIGGINS,NaN,Igualdad,[2ºTiempo]/[60'],NaN,NaN,False,NaN


# Procesamiento

In [ ]:
df_matches = []
matches = df_full["Partido"].unique()

In [ ]:
def local_visitante(teams, match,team): 
    # local y visitante
    pos_guion = match.find("-")
    pos_team = match.find(team)
    oth_team = teams[0] if (teams[0] != team) else teams[1]
    local_team = team if pos_guion > pos_team else oth_team
    visit_team = oth_team if pos_guion > pos_team else team
    return local_team, visit_team
# attackplusratio
columns_kpi = ["AttackPlus", "AttackPossesions", "AttackPlusMinus", "AttackPossesionsRatio", 
               "DefPlus", "DefPossesions", "DefPlusMinus", "DefPossesionsRatio"]
columns_out = ["Match", "Player"] + columns_kpi
df_out = pd.DataFrame(columns = columns_out )
team = "BERA BERA"

In [ ]:
for match in matches: 
    df = df_full[df_full["Partido"] == match]
    df_pm_out = pd.DataFrame(columns = columns_out )
    # match features
    teams = df["Equipo"].unique()
    match = df["Partido"].unique()[0]
    local_team, visit_team = local_visitante(teams, match, team)
    plusminus_att = "Plus Minus Local" if (local_team == team) else "Plus Minus Visitante" 
    oth_team = teams[0] if (teams[0] != team) else teams[1]
    # filtrar posesiones
    df_pm = df[["Equipo","Posesion", "No Posesion", "Periodo", plusminus_att]][df.Posesion.notnull()]
    df_pm = df_pm[df["No Posesion"] == False]
    df_pm = df_pm[df_pm[plusminus_att].notnull()]

    # calculos generales
    df_b = df_pm[df_pm.Equipo == team]
    df_e = df_pm[df_pm.Equipo == oth_team]
    pos_attack = df_b.shape[0]
    goles_marc = df_b[df_b.Posesion =='[Acierto]/[Gol]'].shape[0]
    pos_def = df_e.shape[0]
    goles_enc = df_e[df_e.Posesion =='[Acierto]/[Gol]'].shape[0]
    
    eff_attack  = goles_marc / pos_attack
    eff_def    = 1 - (goles_enc / pos_def)
    
    print(match, pos_attack, goles_marc, pos_def, goles_enc)



    res = 0
    col = ""
    for i, fila in df_pm.iterrows():
        # Compute score
        res = 0
        col = ""
        posesion = fila["Posesion"]
        if (fila["Equipo"] == team) :
            if (posesion.find ("Error") != -1):
                res = 0
            elif (posesion.find("Acierto") != -1):    
                res = 1
            col = "AttackPlus"
            
        else: 
            if (posesion.find ("Error") != -1):
                res = 1
            elif (posesion.find("Acierto") != -1):    
                res = 0
            col = "DefPlus"
        lst_players = fila[plusminus_att].split(",")
        for player in lst_players:
            player = player.strip()
            if (df_pm_out[(df_pm_out.Player == player)].shape[0] == 0):
                df_player = pd.DataFrame([[match, player,0,0,0,0,0,0,0,0]], columns = columns_out)
                df_pm_out = pd.concat([df_player, df_pm_out])
            
            df_pm_out.loc[df_pm_out['Player']== player , col] += res
            df_pm_out.loc[df_pm_out['Player']== player , col.replace("Plus", "") +"Possesions"] += 1        
 

    df_pm_out = df_pm_out.apply(pd.to_numeric, errors='ignore')
    
    df_pm_out["AttackPlusMinus"] = ((df_pm_out["AttackPlus"]/ df_pm_out["AttackPossesions"]) - eff_attack)*100
    df_pm_out["DefPlusMinus"] = ((df_pm_out["DefPlus"]/ df_pm_out["DefPossesions"]) - eff_def)*100
    df_pm_out["AttackPossesionsRatio"] = (df_pm_out["AttackPossesions"] / pos_attack) * 100
    df_pm_out["DefPossesionsRatio"] = (df_pm_out["DefPossesions"] / pos_def) * 100
    
    df_matches.append(df_pm_out)
    
    
df_out = pd.concat(df_matches)

df_out.fillna(0, inplace=True)
df_out[columns_kpi] = df_out[columns_kpi].round()



200822_BER-GRA 72 31 70 30


In [ ]:
from google.colab import files
file_name = df["Partido"].unique()[0] + '_pmi.xlsx'
df_out.to_excel(file_name, columns = columns_out) 
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>